In this notebook, we’ll learn about nonlocal games: a way to validate our quantum mechanical descriptions of the universe with friends using multi-qubit systems.

We dive into a relatively new Python package called `QuTiP` that will allow us to program quantum systems faster and has some cool built-in features for simulating quantum mechanics. Then we’ll learn how to use QuTiP to program a simulator for multiple qubits and see how that changes (or doesn’t!) the three main tasks for our qubits: state preparations, operations, and measurement. This will let us finish the implementation of the the game.

Defining the game:

The `CHSH` game, a nonlocal game with two players and a referee. The referee gives each player a question in the form of a bit value, and then each player has to figure out how to respond to the referee. The players win if the Boolean AND of their responses is the same as the classical XOR of the referee’s questions.

### Install Qutip

In [ ]:
!pip install qutip

     |████████████████████████████████| 14.6 MB 7.3 MB/s 


### Libraries and CHSH game

In [ ]:
# QuTiP’s representation of the Hadamard operation
#
from qutip.qip.operations import hadamard_transform 

H = hadamard_transform()
H

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[ 0.70710678  0.70710678]
 [ 0.70710678 -0.70710678]]

In [ ]:
# Making a Qobj from a vector representing a qubit state 

import qutip as qt 
ket0 = qt.Qobj([[1], [0]])
ket0

Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[1.]
 [0.]]

❶ One key difference between creating `Qobj instances` and arrays is that when we create `Qobj instances`, we always need two levels of lists. The outer list is a list of rows in the new `Qobj instance`. 

❷ QuTiP prints some metadata about the size and shape of the new quantum object, along with the data contained in the new object. In this case, the data for the new Qobj has two rows, each with one column. We identify that as the vector or ket that we use to write the $|0 〉$ state.

In [ ]:
# Using QuTiP to easily create |0 〉 and |1 〉 

ket0 = qt.basis(2, 0)
ket0

Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[1.]
 [0.]]

In [ ]:
ket1 = qt.basis(2, 1)
ket1

Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[0.]
 [1.]]

In [ ]:
# Using QuTiP to create an object for the X matrix
qt.sigmax()

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0. 1.]
 [1. 0.]]

In [ ]:
# Tensor products in QuTiP
#import qutip as qt
#from qutip.qip.operations import hadamard_transform

psi = qt.basis(2, 0) # Sets psi to represent |Ψ 〉 = |0 〉
phi = qt.basis(2, 1) # Sets phi to represent |ϕ 〉 = |1 〉
qt.tensor(psi, phi)

Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[0.]
 [1.]
 [0.]
 [0.]]

In [ ]:
H = hadamard_transform()
I = qt.qeye(2) # We can use the qeye function provided by QuTiP to get a copy of a Qobj instance
print(qt.tensor(H, I))

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.70710678  0.          0.70710678  0.        ]
 [ 0.          0.70710678  0.          0.70710678]
 [ 0.70710678  0.         -0.70710678  0.        ]
 [ 0.          0.70710678  0.         -0.70710678]]


A math trick we can use is to take the left side and subtract from it the right. We should end up with 0. We give this a try in the following cell.

In [ ]:
# Verifying the tensor product in QuTiP 
print((qt.tensor(H, I) * qt.tensor(psi, phi) - qt.tensor(H * psi, I * phi)))

Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[0.]
 [0.]
 [0.]
 [0.]]


❶ The right side of the statement we are trying to prove, where we use $H$ and $I$ as $U$ and $V: (U ⊗ V) (|Ψ 〉 ⊗ |ϕ 〉)$.
 
❷ The left side of the statement we are trying to prove, where we use $H$ and $I$ as $U$ and $V: (U|Ψ 〉) ⊗ (V|ϕ 〉)$.
 
❸ Yay! The two sides of the equation are equal if their difference is 0.

**Upgrading the simulator.** 

The goal now is to use `QuTiP` to upgrade our single-qubit simulator to a multi-qubit simulator with some of the features of `QuTiP`.

In [ ]:
# interface.py: adding a new ry operation
#
class Qubit(metaclass=ABCMeta):

      @abstractmethod
      def h(self):
          pass

      @abstractmethod
      def x(self):
          pass

      @abstractmethod
      def ry(self, angle: float):
          pass # The abstract method ry, which takes an argument angle to specify how far to rotate the qubit around the Y-axis

      @abstractmethod
      def measure(self) -> bool:
          pass

      @abstractmethod
      def reset(self):
          pass

In [ ]:
### simulator.py: Defines a class that implements a multi-qubit simulator.

from interface import QuantumDevice, Qubit
import qutip as qt
from qutip.qip.operations import hadamard_transform
import numpy as np
from typing import List

KET_0 = qt.basis(2, 0)
H = hadamard_transform()

class SimulatedQubit(Qubit):
    qubit_id: int
    parent: "Simulator"

    def __init__(self, parent_simulator: "Simulator", id: int):
        self.qubit_id = id
        self.parent = parent_simulator

    def h(self) -> None:
        self.parent._apply(H, [self.qubit_id])

    def ry(self, angle: float) -> None:
        self.parent._apply(qt.ry(angle), [self.qubit_id])

    def x(self) -> None:
        self.parent._apply(qt.sigmax(), [self.qubit_id])

    def measure(self) -> bool:
        projectors = [
            qt.circuit.gate_expand_1toN(
                qt.basis(2, outcome) * qt.basis(2, outcome).dag(),
                self.parent.capacity,
                self.qubit_id
            )
            for outcome in (0, 1)
        ]
        post_measurement_states = [
            projector * self.parent.register_state
            for projector in projectors
        ]
        probabilities = [
            post_measurement_state.norm() ** 2
            for post_measurement_state in post_measurement_states
        ]
        sample = np.random.choice([0, 1], p=probabilities)
        self.parent.register_state = post_measurement_states[sample].unit()
        return bool(sample)

    def reset(self) -> None:
        if self.measure(): self.x()

class Simulator(QuantumDevice):
    capacity: int
    available_qubits: List[SimulatedQubit]
    register_state: qt.Qobj

    def __init__(self, capacity=3):
        self.capacity = capacity
        self.available_qubits = [
            SimulatedQubit(self, idx)
            for idx in range(capacity)
        ]
        self.register_state = qt.tensor(
            *[
                qt.basis(2, 0)
                for _ in range(capacity)
            ]
        )
    def allocate_qubit(self) -> SimulatedQubit:
        if self.available_qubits:
            return self.available_qubits.pop()

    def deallocate_qubit(self, qubit: SimulatedQubit):
        self.available_qubits.append(qubit)

    def _apply(self, unitary: qt.Qobj, ids: List[int]):
        if len(ids) == 1:
            matrix = qt.circuit.gate_expand_1toN(
                unitary, self.capacity, ids[0]
            )
        else:
            raise ValueError("Only single-qubit unitary matrices are supported.")

        self.register_state = matrix * self.register_state

**Measuring up**

❶ Start by writing $|+ 〉$ as $H|0〉$. In QuTiP, we use the hadamard _transform function to get a `Qobj instance` to represent $H$, and we use $basis(2, 0)$ to get a `Qobj` representing $|0 〉$. 

❷ We can print out ket_plus to get a list of the elements in 
that vector; as before, we call each of these elements an amplitude. 

❸ To represent the state $|+ 〉$, we use that $|+ 〉 = |+ 〉 ⊗ |+ 〉$. 

❹ This output tells us that $|++ 〉$ has the same amplitude for each of the four computational basis states $|00 〉, |01 〉, |10 〉$, and $|11 〉$, just as ket_plus has the same amplitude for each of the computational basis states $|0 〉$ and $|1 〉$.

In [ ]:
# Representing the |++ 〉 state with QuTiP

#import qutip as qt
#from qutip.qip.operations import hadamard_transform
ket_plus = hadamard_transform() * qt.basis(2, 0)
ket_plus

Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[0.70710678]
 [0.70710678]]

In [ ]:
register_state = qt.tensor(ket_plus, ket_plus)
register_state

Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[0.5]
 [0.5]
 [0.5]
 [0.5]]

**Moving on to the game**

To start the quantum strategy, we need to create a `QuantumDevice` instance where we will simulate our qubits.

❷ Labels can be assigned to each qubit as we allocate them to the `shared_system`. 

❸ We cheat a little to set the state of our qubits to the entangled state $(|00 〉 + |11 〉) / √2$. We will see in later on how to prepare this state from scratch and why the function to prepare this state is called bell_state. 

❹ Angles for the rotations we and Eve need to do based on our input from the referee.

❺ Strategy for playing the `CHSH` game starts with us rotating our qubit based on the input classical bit from the referee. 

❻ The classical bit value our strategy returns is the bit value we get when we measure our qubit. 

❼ Eve’s strategy is similar to ours; she just uses different angles for her initial rotation. 

❽ Just like our classical strategy, quantum_strategy returns a tuple of functions that represent our and Eve’s individual actions.

In [ ]:
# Python implementation for the CHSH game

#import random
#from functools import partial
#from typing import Tuple, Callable
#import numpy as np
#from interface import QuantumDevice, Qubit
#from simulator import Simulator

Strategy = Tuple[Callable[[int], int], Callable[[int], int]]

def random_bit() -> int:
    return random.randint(0, 1)

def referee(strategy: Callable[[], Strategy]) -> bool:
    you, eve = strategy()
    your_input, eve_input = random_bit(), random_bit()
    parity = 0 if you(your_input) == eve(eve_input) else 1
    return parity == (your_input and eve_input)

def est_win_probability(strategy: Callable[[], Strategy],
                        n_games: int = 1000) -> float:
    return sum(
        referee(strategy)
        for idx_game in range(n_games)
    ) / n_games

def constant_strategy() -> Strategy:
    return (
        lambda your_input: 0,
        lambda eve_input: 0
    )

import qutip as qt

def quantum_strategy(initial_state: qt.Qobj) -> Strategy:
    shared_system = Simulator(capacity=2)
    shared_system.register_state = initial_state
    your_qubit = shared_system.allocate_qubit()
    eve_qubit = shared_system.allocate_qubit()
    shared_system.register_state = qt.bell_state()
    your_angles = [90 * np.pi / 180, 0]
    eve_angles = [45 * np.pi / 180, 135 * np.pi / 180]

    def you(your_input: int) -> int:
        your_qubit.ry(your_angles[your_input])
        return your_qubit.measure()

    def eve(eve_input: int) -> int:
        eve_qubit.ry(eve_angles[eve_input])
        return eve_qubit.measure()
    return you, eve

if __name__ == "__main__":
    constant_pr = est_win_probability(constant_strategy, 100)
    print(f"Constant strategy won {constant_pr:0.1%} of the time.")

    initial_state = qt.Qobj([[1], [0], [0], [1]]) / np.sqrt(2)
    quantum_pr = est_win_probability(
                    partial(quantum_strategy, initial_state),
                    100
                )
    print(f"Quantum strategy won {quantum_pr:0.1%} of the time " \
          f"with initial state:\n{initial_state}.")

In [ ]:
# Running CHSH with our new quantum_strategy

est_win_probability(quantum_strategy)

❶ You may get slightly more or less than 85% when you try this, because the win probability is estimated under the hood using a binomial distribution.

In [ ]:
### Copyright (c) Sarah Kaiser and Chris Granade.# Code sample from the book "Learn Quantum Computing with Python and Q#" by# Sarah Kaiser and Chris Granade, published by Manning Publications Co.# Book ISBN 9781617296130.# Code licensed under the MIT License.##